**Data extraction for probe questions** 

This code aims to extract usefull information from the experiment's CSV-files. The goal is to retrive the probes and the answers for each subject and session. These needs to be aligned with the time for eeg recordings. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Enable inline plotting for Jupyter Notebook
%matplotlib inline

**Retrieve all dataframes**

In [2]:
# Define the path to the directory containing the CSV files
path = r'/home/sivert/Documents/Specialization_project/Datasets/jin2019/study1/raw/beh/'

# Dict to store dataframes
dataframes = {}

# Generate all the keys for the dict
for i in range(1, 31):
    for j in range(1, 3):
        file_name = f'subject-{i}_s{j}.csv'
        file_path = os.path.join(path, file_name)
        key = f'subject-{i}'
        if os.path.exists(file_path):
            if key not in dataframes:
                dataframes[key] = {}
            dataframes[key][f'session-{j}'] = pd.read_csv(file_path)


**Select relevant columns**

In [3]:
# List of relevant columns
relevant_columns = ['probe_on', 'response_probe_content', 'practice_on', 'block_name']

# Drop all irrelevant columns in the dataframes
for subject, sessions in dataframes.items():
    for session, df in sessions.items():
        dataframes[subject][session] = dataframes[subject][session][relevant_columns]

**Remove sessions with practice on**

In [4]:
# Initialize an empty list to store the subject and session numbers
subject_session_with_practice = []

# Iterate through the dataframes to find the required datasets
for subject, sessions in dataframes.items():
    for session, df in sessions.items():
        if df['practice_on'].eq(1).any():
            subject_number = int(subject.split('-')[1])
            session_number = int(session.split('-')[1])
            subject_session_with_practice.append((subject_number, session_number))

#Print out the subject and session numbers with practice
print('Subjects and sessions with practice on:')
for subject, session in subject_session_with_practice:
    print(f'Subject: {subject}, Session: {session}')

# remove the subjects and sessions with practice and drop the column as it is unnecessary at this point.
for subject, sessions in dataframes.items():
    for session, df in sessions.items():
        if (int(subject.split('-')[1]), int(session.split('-')[1])) in subject_session_with_practice:
            dataframes[subject][session] = dataframes[subject][session][dataframes[subject][session].practice_on.eq(0)]
        dataframes[subject][session] = dataframes[subject][session].drop(columns=['practice_on'])

dataframes['subject-1']['session-1']

Subjects and sessions with practice on:
Subject: 1, Session: 1
Subject: 18, Session: 2


,probe_on,response_probe_content,block_name


**Make task indicator**

In [5]:
import re

# Function to replace block_name content
def replace_block_name(block_name):
    if re.match(r'^vs$', block_name):
        return 0
    elif re.match(r'^sart\d*$', block_name):
        return 1
    else:
        return block_name

# Apply the function to all dataframes
for subject, sessions in dataframes.items():
    for session, df in sessions.items():
        df['block_name'] = df['block_name'].apply(replace_block_name)

In [6]:
dataframes['subject-2']['session-1'].head(20)

,probe_on,response_probe_content,block_name
0,0,NaN,1
1,0,NaN,1
2,0,NaN,1
3,0,NaN,1
4,0,NaN,1
5,0,NaN,1
6,0,NaN,1
7,1,2,1
8,0,2,1
9,0,2,1


**Deal with nan values and strings**

In [7]:
# Function to replace NaN values and ints outside [1,5] with zero and ensure all values are integers
def replace_nan_and_strings(df):
    def convert_and_replace(x):
        if pd.isna(x):
            return 0
        try:
            x = int(x)
        except ValueError:
            return 0
        return x if 1 <= x <= 6 else 0

    df = df.applymap(convert_and_replace)
    return df

# Apply the function to all dataframes
for subject, sessions in dataframes.items():
    for session, df in sessions.items():
        dataframes[subject][session] = replace_nan_and_strings(df)

# Display the updated dataframe for subject-3 session-1
dataframes['subject-3']['session-1']

/tmp/ipykernel_715367/1055512194.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(convert_and_replace)


,probe_on,response_probe_content,block_name
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
820,0,3,1
821,0,3,1
822,0,3,1
823,0,3,1


In [8]:
# Display the updated dataframe for subject-1 session-1
dataframes['subject-2']['session-1']

# show rows with response_probe_content equal to 0
dataframes['subject-2']['session-1'][dataframes['subject-2']['session-1'].response_probe_content.eq(0)]

,probe_on,response_probe_content,block_name
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
5,0,0,1
6,0,0,1
26,1,0,1
27,0,0,1
28,0,0,1


**Export new event data**

In [9]:
# Define the path to the new directory
new_path = os.path.join(path, 'new_events')

# Create the directory if it does not exist
if not os.path.exists(new_path):
    os.makedirs(new_path)

# Export each dataframe to a CSV file in the new directory
for subject, sessions in dataframes.items():
    for session, df in sessions.items():
        file_name = f'{subject}_{session}_events.csv'
        file_path = os.path.join(new_path, file_name)
        df.to_csv(file_path, index=False)

In [10]:
import pandas as pd
import os

def load_dataframes(path):
    dataframes = {}
    for i in range(1, 31):
        for j in range(1, 3):
            file_name = f'subject-{i}_session-{j}_events.csv'
            file_path = os.path.join(path, file_name)
            key = f'subject-{i}'
            if os.path.exists(file_path):
                if key not in dataframes:
                    dataframes[key] = {}
                dataframes[key][f'session-{j}'] = pd.read_csv(file_path)
    return dataframes

# Define the path to the directory containing the CSV files
path = '/home/sivert/Documents/Specialization_project/Datasets/jin2019/study1/raw/beh/new_events'

# Load the dataframes
dataframes = load_dataframes(path)

# Example usage: Accessing a specific dataframe
print(dataframes['subject-2']['session-1'])

     probe_on  response_probe_content  block_name
0           0                       0           1
1           0                       0           1
2           0                       0           1
3           0                       0           1
4           0                       0           1
..        ...                     ...         ...
820         0                       3           0
821         1                       1           0
822         0                       1           0
823         0                       1           0
824         0                       1           0

[825 rows x 3 columns]
